<a href="https://colab.research.google.com/github/BrownParticleAstro/dmtools/blob/podman-python-dash/notebooks/DMTOOLSGraphDemo_withparser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [32]:
!pip install dash

In [33]:
!pip install --upgrade certifi

In [34]:
import ssl
import urllib
context = ssl._create_unverified_context()
from urllib.request import Request, urlopen

In [35]:
import dash
from dash import Dash
import json
import requests
import pathlib
import sqlalchemy
from sqlalchemy import text
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dash_table
from dash import dcc, callback, html
from dash import Input, Output, State

from itertools import cycle

from decimal import *

In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Decimal was to maintain accuracy of cross section data

In [37]:
def setup_decimal_precision():
    getcontext().prec = 24
    print(Decimal(748327402479023).sqrt())
setup_decimal_precision()

27355573.5176402945763287


In [38]:
# Build App
# app = Dash(__name__)

In [39]:
# colors
# palette = cycle(px.colors.qualitative.Bold)

# Set Up Request Header

In [40]:
#request_header = {'dmtool-userid': '1' ,'dmtool-apikey': 'CC0EF1DD_C9AE_11EE_BCAD_0EC372396BDC'} # dev server
#request_header = {'dmtool-userid': '1' ,'dmtool-apikey': '81D34E22_CC0F_11EE_8826_FE852FEDCE00'} ## prod server
# 58E88A3E_37DC_11EF_BCE0_FADBD26D994C

def create_request_header(dmtool_userid_in,dmtool_apikey_in ):
    request_header_out = {'dmtool-userid': dmtool_userid_in ,'dmtool-apikey': dmtool_apikey_in}
    return request_header_out
request_header = create_request_header('1','58E88A3E_37DC_11EF_BCE0_FADBD26D994C' )

# Data Introduction

    data is the Raw Upload Format
    data_data is the parsed data
    data_about is the summary fields only
    data_display is a record for each data series in the dataset and how it appears in a chart
    data_display contains the data to be shown in each plot (previously known as limit_display)

    plot is a record for each plot and holds a title and axis ranges

    data, data_data, data_about, data_display, plot are all data_subjects

    by using data_subjects this approach can easily be extended

# API Design

The api have been designed to retrieve data based on:
    plot - id of the plot
    data - id of the original dataset in data
    id - each record is uniquely numbered

# Route Naming

The routes have been named as follows

    action + how/what + subject

    read_plot_data_display

This makes understanding code easier and allowed for dynamic creation.

# Every table has full CRUD functionality (action)

  C - Create

    create_
  R - Read

    read_all_ --- read all open and owned records
    read_plot_ --- read all records relating to a plot
    read_data_ --- read all records relating to a specific dataset
  U - Update

    update_ --- update a specific record
  D - Delete

    delete_ --- delete a specific record

  A - Archive

    archive_ --- archive a record so it cannot be used in new plots

# Read All not plural

Plural subjects for read_all seemed unecessary.

e.g. read_all_data_display

# Set Up Request URL

In [41]:
# this demonstrates the full url for the api services
# only open api are available to external users
# two api servers are being used
  # data is the external api services
  # about is the internal security api services like login and api key

api_server = "https://dmtools.het.brown.edu/"
data_api = "dmtool/fastapi_data/open/data/"

fastapi_url_data_data = api_server + data_api + "read_all_data_data/"

fastapi_url_read_plot_data_data = api_server + data_api + "read_plot_data_data/"

fastapi_url_data_about = api_server + data_api + "read_all_data_about"
fastapi_url_read_all_data_display = api_server + data_api +  "read_all_data_display/"

fastapi_url_read_plot_data_display = api_server + data_api +  "read_plot_data_display/"


fastapi_url_plots = api_server + data_api + "read_all_plot/"
#/dmtool/fastapi_data/open/data/plots

In [46]:
fastapi_url_data = api_server + data_api + "read_a_data/" + "?id_in=1"
fastapi_url_data

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_a_data/?id_in=1'

In [47]:
request = Request(fastapi_url_data, headers=request_header)
r = urllib.request.urlopen(request, context=context)

#r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
#print(r.read())
string = r.read().decode('utf-8')
data_json_obj = json.loads(string)
#all_plots_json_obj[1]
data_df = pd.DataFrame(data_json_obj)
data_df['row_id'] = data_df.index

In [50]:
data_df[['id', 'subject', 'data_values', 'data_label']]

,id,subject,data_values,data_label
0,1,data,{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;63...,"Baer et. al., 2003, constraints on mSUGRA, SI"


In [52]:
data_data = []
row_id = 0
for index, row in data_df.iterrows():
    id = row['id']
    print('id from data >>>>', id)
    data_id = row['id']
    data_label = row['data_label']
    data_string = row['data_values']

    data_string = data_string.replace("{[", "")
    data_string = data_string.replace("]}", "")

    data_series = data_string.split("]")

    for l in range(0,len(data_series)):
        series_id = 0
        trace_id = l + 1
        try:
            trace_name = str(l + 1) + "_" + row['data_label']
        except:
            trace_name = str(l + 1) + "_" + 'trace_name'
        data_label = row['data_label']
        subject = 'data_data'
        single_set = data_series[l]
        set_list = single_set.split(";")
        for i in set_list:
            ## the following was added due to a different approach to data_string format
            r0 = i.replace(',[','')
            r1 = r0.replace('  ',' ')
            r2 = r1.replace('  ',' ')
            r3 = r2.replace('  ',' ')
            r4 = r3.replace('\r\n','')
            r5 = r4.replace('\t',' ')
            r6 = r5.replace(',',' ')
            r7 = r6.replace('  ',' ')
            r8 = r7.replace('  ',' ')
            s = r8.lstrip()
            z = s.split(" ");
            try:
                raw_y = z[1]
                raw_x = z[0].replace(",[", "")
            except:
                raw_y = '0'
                raw_x = '0'

            try:
                masses =  Decimal(raw_x)
                #cross_sections = float(raw_y)/scale_factor
                raw_y_l = raw_y.split('e')
                raw_y_n = raw_y_l[0]
                try:
                    raw_y_e = raw_y_l[1]
                except:
                    raw_y_e = 0
                cross_sections = Decimal(raw_y)
                cross_sections_n = Decimal(raw_y_n)
                cross_sections_e = Decimal(raw_y_e)
                if cross_sections == 0:
                    print("this raw_y gave 0 >>>>" , raw_y)
            except:
                masses = 0
                cross_sections = 0
            try:
                appendthis = [subject,data_id,row_id,series_id,trace_id,trace_name,data_label,raw_x,raw_y,masses,cross_sections,cross_sections_n,cross_sections_e]
            except:
                appendthis = [subject,data_id,row_id,limit_id,series_id,trace_id,trace_name,data_label,'0','0','0','0','0','0']

            data_data.append(appendthis)
            row_id += 1
            series_id += 1

    keys = ['subject','data_id','row_id','limit_id','series_id','trace_id','trace_name','data_label','raw_x','raw_y','masses','cross_sections','cross_sections_n','cross_sections_e']
    data_data_lod = [dict(zip(keys, l)) for l in data_data ]
    '''
    for record in data_data_lod:
        subject = 'data_data',
        row_id = record['row_id']
        limit_id = record['limit_id']
        series_id = record['series_id']
        data_id = record['data_id']
        trace_id = record['trace_id']
        raw_x = record['raw_x']
        raw_y = record['raw_y']
        masses = record['masses']
        cross_sections_n = record['cross_sections_n']
        cross_sections_e = record['cross_sections_e']
    '''

id from data >>>> 1


In [53]:
data_data_lod

[{'subject': 'data_data',
  'data_id': 1,
  'row_id': 0,
  'limit_id': 0,
  'series_id': 1,
  'trace_id': '1_Baer et. al., 2003, constraints on mSUGRA, SI',
  'trace_name': 'Baer et. al., 2003, constraints on mSUGRA, SI',
  'data_label': '60',
  'raw_x': '4e-43',
  'raw_y': Decimal('60'),
  'masses': Decimal('4E-43'),
  'cross_sections': Decimal('4'),
  'cross_sections_n': Decimal('-43')},
 {'subject': 'data_data',
  'data_id': 1,
  'row_id': 1,
  'limit_id': 1,
  'series_id': 1,
  'trace_id': '1_Baer et. al., 2003, constraints on mSUGRA, SI',
  'trace_name': 'Baer et. al., 2003, constraints on mSUGRA, SI',
  'data_label': '70',
  'raw_x': '4e-43',
  'raw_y': Decimal('70'),
  'masses': Decimal('4E-43'),
  'cross_sections': Decimal('4'),
  'cross_sections_n': Decimal('-43')},
 {'subject': 'data_data',
  'data_id': 1,
  'row_id': 2,
  'limit_id': 2,
  'series_id': 1,
  'trace_id': '1_Baer et. al., 2003, constraints on mSUGRA, SI',
  'trace_name': 'Baer et. al., 2003, constraints on mSUGR

In [ ]:
'''


def fill_color_rgba(color_in):
if color_in == 'cyan':
    color_out = 'rgba(0,255,255,0.05)'
elif color_in == 'red':
    color_out = 'rgba(255,0,0,0.05)'
elif color_in == 'blue':
    color_out = 'rgba(0,0,255,0.05)'
elif color_in == 'green':
    color_out = 'rgba(0,255,0,0.05)'
elif color_in == 'black':
    color_out = 'rgba(0,0,0,0.05)'
elif color_in == 'magenta':
    color_out = 'rgba(255,0,255,0.05)'
elif color_in == 'yellow':
    color_out = 'rgba(255,255,0,0.05)'
elif color_in == 'white':
    color_out = 'rgba(255,255,255,0.05)'
else:
    color_out = 'rgba(255,0,0,0.05)'
return color_out


@router.post("/populate_data_display_for_plot/{command_in}")
async def populate_data_display_for_plot(command_in: str,
                                                session: AsyncSession = Depends(get_session),
                                                dmtool_userid: Annotated[int | None, Header()] = None):
##command_in = 'all'

batch1_list = [53,66,73,107,148,165,183,192,250,262,263,269,272,301,313,327,367,374,377,454,487,520,524,553,561,570,622,\
                650,694,698,709,723,724,775,776,777,813,814,816,912,933,953,959,995,1124,1214,1215,1216,1217,1219,1454,1485,\
                1486,1612,1623,1647,1746,1747,1858,1859,2032,2060,2061,2069,2070,2142,2152,2153,2154,2155]
batch2_list = [2156,2164,2165,2166,2167,2168,2169,2170,2171,2180,2186,2189,2217,2229,2247,2252,2255,2256,2257,2276,2277,2295,\
                2301,2387,2428,2439,2466,2535,2536,2564,2565,2570,2572,2660,2745,2812,2828,2841,3013,3019,3020,3152,3180,3208,\
                3360,3378,3450,3468,3494,3520,3647,3714,3717,3772,3816,3850,3967,3968,3969,4042,4131,4373,4374,4396,4523,4643,4644]
if command_in == 'batch1':
    list_of_old_plots = batch1_list
elif command_in == 'batch2':
    list_of_old_plots = batch2_list
else:
    list_of_old_plots = [53]

all_limit_display = select(Limit_display).where(Limit_display.plot_id.in_(list_of_old_plots))
all_limit_display_session = await session.exec(all_limit_display)
all_limit_display_result = all_limit_display_session.all()

#list_of_old_plots = []
#for ald in all_limit_display_result:
#    this_dict = ald.__dict__
#    plot_id = this_dict['plot_id']
#    if plot_id not in list_of_old_plots:
#        list_of_old_plots.append(plot_id)


for lop in list_of_old_plots:
    #statement = delete(User).where(User.id.in_(...))
    #session.execute(statement)
    statement = delete(Data_display).where(Data_display.old_plot_id == lop)
    #try:
    #results = await session.exec(statement)
    #data_display = results.all()
    await session.execute(statement)
    await session.commit()
    #except:
    #    pass
    statement = select(Limit_display).where(Limit_display.plot_id == lop)
    results = await session.exec(statement)
    limit_displays = results.all()
    print(lop)
    for ld in limit_displays:
        this_dict = ld.__dict__
        limit_id = this_dict['limit_id']

        new_plot = select(Plot).where(Plot.old_plot_id == lop)
        new_plot_session = await session.exec(new_plot)
        new_plot_result = new_plot_session.first()
        new_plot_dict = new_plot_result.__dict__
        new_plot_id = new_plot_dict['id']

        ld_style = this_dict['style']

        ## fill style

        if ld_style == "dot":
            f_style = "none"
        elif ld_style == "dash":
            f_style = "none"
        elif ld_style ==  "fill":
            f_style =  "fill"
        elif ld_style ==  "line":
            f_style = "none"
        elif ld_style == "point":
            f_style = "none"
        elif ld_style ==  "cross":
            f_style = "none"
        elif ld_style ==  "circle":
            f_style = "none"
        elif ld_style ==  "plus":
            f_style = "none"
        elif ld_style ==  "asterisk":
            f_style = "none"
        elif ld_style ==  "pent":
            f_style = "none"
        elif ld_style == "hex":
            f_style = "none"
        elif ld_style == "triu":
            f_style = "none"
        elif ld_style ==  "trid":
            f_style = "none"
        elif ld_style ==  "tril":
            f_style = "none"
        elif ld_style ==  "trir":
            f_style = "none"
        else :
            f_style = ld_style

        ## line style

        if ld_style == "dotted":
            l_style = "dotted"
        elif ld_style == "dash":
            l_style = "dash"
        elif ld_style ==  "fill":
            l_style =  "solid"
        elif ld_style ==  "line":
            l_style = "solid"
        elif ld_style == "point":
            l_style = "none"
        elif ld_style ==  "cross":
            l_style = "none"
        elif ld_style ==  "circle":
            l_style = "none"
        elif ld_style ==  "plus":
            l_style = "none"
        elif ld_style ==  "asterisk":
            l_style = "none"
        elif ld_style ==  "pent":
            l_style = "none"
        elif ld_style == "hex":
            l_style = "none"
        elif ld_style == "triu":
            l_style = "none"
        elif ld_style ==  "trid":
            l_style = "none"
        elif ld_style ==  "tril":
            l_style = "none"
        elif ld_style ==  "trir":
            l_style = "none"
        else :
            l_style = ld_style

          ## symbol style

        if ld_style == "dotted":
            sym_style = "none"
        elif ld_style == "dash":
            sym_style = "none"
        elif ld_style ==  "fill":
            sym_style =  "none"
        elif ld_style ==  "line":
            sym_style = "none"
        elif ld_style == "point":
            sym_style = "circle"
        elif ld_style ==  "cross":
            sym_style = "cross"
        elif ld_style ==  "circle":
            sym_style = "circle"
        elif ld_style ==  "plus":
            sym_style = "plus"
        elif ld_style ==  "asterisk":
            sym_style = "star"
        elif ld_style ==  "triangle":
            sym_style = "triangle"
        elif ld_style == "hex":
            sym_style = "triangle"
        elif ld_style == "triu":
            sym_style = "triangle"
        elif ld_style ==  "trid":
            sym_style = "triangle"
        elif ld_style ==  "tril":
            sym_style = "triangle"
        elif ld_style ==  "trir":
            sym_style = "triangle"
        else :
            sym_style = ld_style

        ## color conversion

        ld_color = this_dict['color']

        if ld_color == "k":
            line_color = 'black'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'black'
        elif ld_color == "r":
            line_color = 'red'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'red'
        elif ld_color ==  "dkg":
            line_color = 'green'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'green'
        elif ld_color ==  "ltg":
            line_color = 'green'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'green'
        elif ld_color == "ltr":
            line_color = 'red'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'red'
        elif ld_color ==  "b":
            line_color = 'blue'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'blue'
        elif ld_color ==  "ltb":
            line_color = 'blue'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'blue'
        elif ld_color ==  "c":
            line_color = 'cyan'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'cyan'
        elif ld_color in ('g10','g20','g30','g40','g50','g60','g70','g80','g90'):
            line_color = 'green'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'green'
        elif ld_color ==  "dkb":
            line_color = 'blue'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'blue'
        elif ld_color == "dkr":
            line_color = 'red'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'red'
        elif ld_color == "g":
            line_color = 'green'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'green'
        elif ld_color == "m":
            line_color = 'magenta'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'magenta'
        elif ld_color == "y":
            line_color = 'yellow'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'yellow'
        elif ld_color == "w":
            line_color = 'white'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'white'
        else :
            line_color = 'purple'
            fill_color = fill_color_rgba(line_color)
            marker_color = 'purple'


        all_data_display = select(Data_display).where(Data_display.limit_id == limit_id).where(Data_display.plot_id == -1)
        all_data_display_session = await session.exec(all_data_display)
        all_data_display_result = all_data_display_session.all()
        for dd in all_data_display_result:
            dd_dict = dd.__dict__
            dd_dict['old_plot_id'] = lop
            data_display = Data_display(
                      data_id = dd_dict['data_id'],
                      old_plot_id = dd_dict['old_plot_id'],
                      subject = 'data_display',
                      limit_id = dd_dict['limit_id'],
                      plot_id = new_plot_id,
                      data_label = dd_dict['data_label'],
                      trace_id = dd_dict['trace_id'],
                      trace_name = dd_dict['trace_name'],
                      line_color = line_color,
                      fill_color = fill_color,
                      line = dd_dict['line'],
                      symbol = sym_style,
                      mode_value = dd_dict['mode_value'],
                      line_width = dd_dict['line_width'],
                      line_format = l_style,
                      fill_style = f_style,
                      opacity_style = dd_dict['opacity_style'],
                      marker_size = dd_dict['marker_size'],
                      marker_color = marker_color
            )

            plot_id : Optional[int] = Field(default=None, nullable=True, primary_key=False)
            series_id : Optional[int] = Field(default=None, nullable=True, primary_key=False)
            data_label : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            x_units : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            y_units : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            x_rescale : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            y_rescale : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            trace_id : Optional[int] = Field(default=None, nullable=True, primary_key=False)
            trace_name : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            line_color : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            symbol_color : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            fill_color : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            line : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            symbol : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            spin_dependency : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            result_type : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            mode_value : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            line_width : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            line_format : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            fill_style : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            opacity_style : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            marker_size : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            marker_color : Optional[str] = Field(default=None, nullable=True, primary_key=False)
            created_at : Optional[datetime] = Field(default=datetime.utcnow(), nullable=True, primary_key=False)
            updated_at : Optional[datetime] = Field(default=datetime.utcnow(), nullable=True, primary_key=False)
            archived_at : Optional[datetime] = Field(default=datetime_origin, nullable=True, primary_key=False)

            session.add(data_display)
            await session.commit()

return 'finished'
'''

In [11]:
#fastapi_url_data_data

In [12]:
#print(fastapi_url_plots)

In [13]:
## List All Plots Available

## List All Plots Available

In [14]:
def get_all_owned_and_open_plots(fastapi_url_plots_in, request_header_in):
    #r = requests.get(fastapi_url_plots, headers = request_header)
    request = Request(fastapi_url_plots_in, headers=request_header)
    r = urllib.request.urlopen(request, context=context)

    #r = urllib.request.urlopen(fastapi_url_plots,context=context, headers = request_header)
    #print(r.read())
    string = r.read().decode('utf-8')
    all_plots_json_obj = json.loads(string)
    #all_plots_json_obj[1]
    all_plots_df_ret = pd.DataFrame(all_plots_json_obj)
    all_plots_df_ret['row_id'] = all_plots_df_ret.index
    #all_plots_df.head(10)
    return all_plots_df_ret
all_plots_df = get_all_owned_and_open_plots(fastapi_url_plots, request_header)
all_plots_df.columns

Index(['x_max', 'updated_at', 'y_min', 'archived_at', 'subject', 'y_max',
       'plot_png', 'id', 'x_units', 'legend_png', 'old_plot_id', 'y_units',
       'plot_eps', 'open', 'user_id', 'legend_eps', 'name', 'old_user_id',
       'no_id', 'x_min', 'created_at', 'row_id'],
      dtype='object')

In [15]:
all_plots_df[['id', 'old_plot_id', 'y_units','open', 'user_id','name', 'created_at', 'row_id']].head(5)

,id,old_plot_id,y_units,open,user_id,name,created_at,row_id
0,20,53,cm2,1,None,XENON10_for_thesis,2009-10-14T07:48:53,0
1,26,66,cm2,1,None,XENON10_LUX_compare,2009-11-19T14:22:11,1
2,33,73,cm2,1,None,XENON10_LUX_ZEPLINIII,2009-11-23T21:12:03,2
3,54,107,cm2,1,None,CDMS_new_results,2010-01-04T18:59:18,3
4,74,148,cm2,1,None,CoGeNT_plus_Xenon_100_New,2010-02-05T00:29:07,4


In [16]:
def GetPlotsContaining(string_in):
    return_df = all_plots_df[['id', 'old_plot_id', 'y_units','open', 'user_id','name', 'created_at']][all_plots_df['name'].str.contains(string_in)].copy()
    return_df['created_at'] = pd.to_datetime(return_df['created_at'], errors='coerce')
    return_df['created_at'] = return_df['created_at'].dt.strftime('%Y%m%d%H%M')
    return return_df
relevant_plots =   GetPlotsContaining('CDMS')
relevant_plots

,id,old_plot_id,y_units,open,user_id,name,created_at
3,54,107,cm2,1,None,CDMS_new_results,201001041859
6,97,183,cm2,1,None,CDMS II limits,201002280018
8,138,250,cm2,1,None,"ZEPLIN-III, XENON10, CDMS",201004201202
11,156,269,cm2,1,None,LUX-XENON100-SuperCDMS,201005032038
12,159,272,cm2,1,None,SuperCDMS-XENON100-LUX,201005041733
13,185,301,cm2,1,None,CDMSII+XENON10 SI,201005291759
14,195,313,cm2,1,None,CDMS II Limit,201006102148
16,232,367,cm2,1,None,"CDMS II (Soudan) SI & SD limits, Ge, Dec. 2010",201102230341
17,238,374,cm2,1,None,"CDMS I (SUF) limits, Ge+Si and Si only",201102240104
18,241,377,cm2,1,None,"CDMS I (SUF) Si+Ge and Si only, 1 to 5 GeV (20...",201102240250


In [17]:
def get_new_plot_id_from_old_id(all_plots_df_in, old_plot_id_in):
    old_plot_details_ret = all_plots_df[['name','x_units', 'id', 'y_units','old_plot_id',  'x_min', 'x_max', 'y_min', 'y_max']] \
    [all_plots_df['old_plot_id']==old_plot_id_in]
    return old_plot_details_ret
old_plot_details = get_new_plot_id_from_old_id(all_plots_df,73)
old_plot_details

,name,x_units,id,y_units,old_plot_id,x_min,x_max,y_min,y_max
2,XENON10_LUX_ZEPLINIII,GeV/c^2,33,cm2,73,1,10000,-48,-32


## Enter Plot ID Here

In [18]:
plot_id = 33
this_plot = all_plots_df[all_plots_df['id']==plot_id].copy()
#this_plot.columns
this_plot['created_at'] = pd.to_datetime(this_plot['created_at'], errors='coerce')
this_plot['created_at'] = this_plot['created_at'].dt.strftime('%Y%m%d%H%M')
this_plot['updated_at'] = pd.to_datetime(this_plot['updated_at'], errors='coerce')
this_plot['updated_at'] = this_plot['updated_at'].dt.strftime('%Y%m%d%H%M')
this_plot[['name','x_max', 'updated_at', 'y_min', 'subject', 'y_max',
       'id', 'x_units', 'old_plot_id', 'y_units',
       'open', 'user_id', 'name',
       'x_min', 'created_at', 'row_id']]

,name,x_max,updated_at,y_min,subject,y_max,id,x_units,old_plot_id,y_units,open,user_id,name,x_min,created_at,row_id
2,XENON10_LUX_ZEPLINIII,10000,201001041903,-48,plot,-32,33,GeV/c^2,73,cm2,1,None,XENON10_LUX_ZEPLINIII,1,200911232112,2


# Extract all Data About Plot into Variables



In [19]:
def get_data_about_plot(all_plots_df_in,plot_id_in):
    this_plot = all_plots_df_in[all_plots_df_in['id']==plot_id_in]
    #this_plot
    start_x_range_ret = int(this_plot.iloc[0]['x_min'])
    stop_x_range_ret = int(this_plot.iloc[0]['x_max'])
    start_y_range_ret = int(this_plot.iloc[0]['y_min'])
    stop_y_range_ret = int(this_plot.iloc[0]['y_max'])
    all_plots_df_in[all_plots_df_in['id'] == plot_id_in][['id','old_plot_id','name']]
    plot_name_df = this_plot[['name','old_plot_id']]
    plot_name_ret = plot_name_df.iloc[0]['name']
    old_plot_id_ret = plot_name_df.iloc[0]['old_plot_id']
    return start_x_range_ret,stop_x_range_ret, start_y_range_ret, stop_y_range_ret, plot_name_ret, old_plot_id_ret

start_x_range,stop_x_range, start_y_range, stop_y_range, plot_name, old_plot_id = get_data_about_plot(all_plots_df, plot_id)



## Get Display Data for Plot (Limit Displays)

In [20]:
#fastapi_url_data_display

In [21]:
def get_data_displays_for_plot(fastapi_url_data_display_in, plot_id_in, request_header_in):
    request = Request(fastapi_url_read_plot_data_display + '?id_in=' + str(plot_id_in), headers=request_header_in)
    r = urllib.request.urlopen(request, context=context)
    string = r.read().decode('utf-8')
    data_display_json_obj = json.loads(string)
    data_display_df_ret = pd.DataFrame(data_display_json_obj)
    data_display_df_ret['row_id'] = data_display_df_ret.index
    data_display_df_ret['updated_at'] = pd.to_datetime(data_display_df_ret['updated_at'], errors='coerce')
    data_display_df_ret['updated_at'] = data_display_df_ret['updated_at'].dt.strftime('%Y%m%d%H%M')
    return data_display_df_ret
data_display_df = get_data_displays_for_plot(fastapi_url_read_plot_data_display, plot_id, request_header)
data_display_df[['id', 'subject','data_id', 'plot_id', 'data_label', 'line_color', 'symbol_color',
       'fill_color', 'line', 'symbol',
       'mode_value', 'line_width', 'line_format', 'fill_style',
       'opacity_style', 'marker_size', 'marker_color', 'updated_at', 'row_id']].head(5)

,id,subject,data_id,plot_id,data_label,line_color,symbol_color,fill_color,line,symbol,mode_value,line_width,line_format,fill_style,opacity_style,marker_size,marker_color,updated_at,row_id
0,3495,data_display,255,33,"XENON10, 2007, first results, 58.6 live days, SI",blue,None,"rgba(0,0,255,0.05)",None,none,None,None,solid,none,None,None,blue,202406201322,0
1,3496,data_display,134,33,LUX 300 kg LXe Projection (Jul 2007) - Offline,cyan,None,"rgba(0,255,255,0.05)",None,none,None,None,dash,none,None,None,cyan,202406201322,1
2,3497,data_display,134,33,LUX 300 kg LXe Projection (Jul 2007) - Offline,cyan,None,"rgba(0,255,255,0.05)",None,none,None,None,dash,none,None,None,cyan,202406201322,2
3,3498,data_display,267,33,"ZEPLIN III, 2009, first science run, 847kg-day...",green,None,"rgba(0,255,0,0.05)",None,none,None,None,solid,none,None,None,green,202406201322,3


In [22]:
#data_display_df = data_display_df[data_display_df.data_id==5]

## Get Data for the Plot (Detailed Data Parsed into data_data table)

In [23]:
fastapi_url_read_plot_data_data + '?id_in=' + str(plot_id)

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_plot_data_data/?id_in=33'

## Convert Text Scientific Numbers into Actual Scientific Numbers

In [24]:
def convert_string_to_decimal(string_in):
  try:
     decimal_out = Decimal(string_in)
  except:
     if string_in == None:
        decimal_out = Decimal(0)
     else:
        decimal_out = Decimal(0)
  return decimal_out

In [25]:
fastapi_url_read_plot_data_data

'https://dmtools.het.brown.edu/dmtool/fastapi_data/open/data/read_plot_data_data/'

In [26]:
def get_data_data_for_plot(fastapi_url_data_data_in, plot_id_in, request_header_in):
    request = Request(fastapi_url_data_data_in + '?id_in=' + str(plot_id_in), headers=request_header_in)
    r = urllib.request.urlopen(request, context=context)

    string = r.read().decode('utf-8')
    data_data_json_obj = json.loads(string)
    # the dtype string was to ensure all the significant figures are brought into the workbook
    # the x y data is also stored in the database as columns, but we could not ensure the data was extracted and sent fully by fastapi
    data_data_df_ret = pd.DataFrame(data_data_json_obj, columns=['data_id','trace_id','series_id','raw_x', 'raw_y', 'x_rescale', 'y_rescale'], dtype='str')
    #data_data_df.dtypes
    data_data_df_ret[['raw_y_n', 'raw_y_e']] = data_data_df_ret['raw_y'].str.split('e', n=1, expand=True)
    #data_data_df['cross_sections_n'] = data_data_df['raw_y_n'].apply(Decimal)
    # convert text to scientific values
    data_data_df_ret['cross_sections_n'] = data_data_df_ret['raw_y_n'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret))
    data_data_df_ret['cross_sections_e'] = data_data_df_ret['raw_y_e'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret))
    # rescale numbers if required
    data_data_df_ret['x_rescale_value'] = data_data_df_ret['x_rescale'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret)) ## cross section
    data_data_df_ret['y_rescale_value'] = data_data_df_ret['y_rescale'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret)) ## mass
    # calculates cross sections and masses
    data_data_df_ret['cross_sections'] = data_data_df_ret['cross_sections_n'] * (10 ** data_data_df_ret['cross_sections_e']) * (10 ** data_data_df_ret['y_rescale_value'])
    data_data_df_ret['masses'] = data_data_df_ret['raw_x'].apply(lambda data_data_df_ret : convert_string_to_decimal(data_data_df_ret)) * (10** data_data_df_ret['x_rescale_value'])
    # convert other columns to int
    data_data_df_ret['data_id'] = data_data_df_ret['data_id'].astype(int)
    data_data_df_ret['trace_id'] = data_data_df_ret['trace_id'].astype(int)
    data_data_df_ret['series_id'] = data_data_df_ret['series_id'].astype(int)

    #data_data_df_ret[['data_id','trace_id','series_id','masses','cross_sections']].head(10)
    return data_data_df_ret

data_data_df = get_data_data_for_plot(fastapi_url_read_plot_data_data, plot_id, request_header)
data_data_df.head(5)

,data_id,trace_id,series_id,raw_x,raw_y,x_rescale,y_rescale,raw_y_n,raw_y_e,cross_sections_n,cross_sections_e,x_rescale_value,y_rescale_value,cross_sections,masses
0,134,1,0,10.593,5.9693e-44,1,1,5.9693,-44,5.9693,-44,1,1,5.96930E-43,105.930
1,134,1,1,11.22,2.6973e-44,1,1,2.6973,-44,2.6973,-44,1,1,2.69730E-43,112.20
2,134,1,2,11.885,1.4709e-44,1,1,1.4709,-44,1.4709,-44,1,1,1.47090E-43,118.850
3,134,1,3,12.589,8.9193e-45,1,1,8.9193,-45,8.9193,-45,1,1,8.91930E-44,125.890
4,134,1,4,13.335,5.8066e-45,1,1,5.8066,-45,5.8066,-45,1,1,5.80660E-44,133.350


## Looking for Min and Max Values for Axes (Requires help)

In [27]:
from math import log10, floor
def find_exp(number_in):
  try:
    base10 = log10(abs(number_in))
  except:
    base10 = log10(1)

  return floor(base10)


#data_data_df.head(5)
min_cross_sections = data_data_df['cross_sections'].min()
max_cross_sections = data_data_df['cross_sections'].max()
min_masses = data_data_df['masses'].min()
max_masses = data_data_df['masses'].max()

print('min y :' ,min_cross_sections)
print('max y :' ,max_cross_sections)
print('min x :' ,min_masses)
print('max x :' ,max_masses)

print(start_x_range)
print(stop_x_range)
print(start_y_range)
print(stop_y_range)

min y : 4.30440E-45
max y : 1.9720E-39
min x : 70
max x : 100000
1
10000
-48
-32


## Creating Blank Chart (This will be used in the app)

In [28]:
def create_blank_chart_for_plot(plot_name_in, plot_id_in, old_plot_id_in):
    y_title_text = r"$\text{WIMP Mass [GeV}/c^{2}]$"
    x_title_text = r"$\text{Cross Section [cm}^{2}\text{] (normalized to nucleon)}$"
    plot_title = plot_name_in + " - Plot Reference:" + str(plot_id_in) + " - Old Plot ID:" + str(old_plot_id_in)
    #plot_title
    ## create empty chart
    fig_chart_out = go.Figure(data=[go.Scatter(x=[], y=[])])
    fig_chart_out.update_layout( autosize=False, width=800, height=800, )
    fig_chart_out.update_layout(xaxis_range=[-1,-4])
    fig_chart_out.update_layout(yaxis_range=[-1,-4])
    fig_chart_out.update_layout(
            margin=dict(l=0, r=0, t=20, b=0),
            paper_bgcolor="LightSteelBlue",)

    fig_chart_out.update_layout(
        title=dict(text=plot_title , font=dict(size=18), automargin=True, yref='paper')
    )
    fig_chart_out.update_xaxes(
        title_text=x_title_text,
        #type="log"
        type="linear"
    )
    fig_chart_out.update_yaxes(
        title_text=y_title_text,
        type="log"
        #type="linear"
    )
    return fig_chart_out

fig_chart = create_blank_chart_for_plot(plot_name,plot_id, old_plot_id)
fig_chart.show()


## Create Actual Chart

In [29]:
def create_actual_plot(plot_name_in, plot_id_in, old_plot_id_in, data_display_df_in, data_data_df_in):
    ## requirement to keep plots square
    #if screen_height_in > screen_width_in:
    #    plot_square_dimensions =
    #plot_square_dimensions = self.screen_height / 2
    plot_square_dimensions = 600

    m1 = go.layout.Margin(l=20,r=10,b=20,t=20,pad=0)
    hw = go.Layout(autosize=False,width=plot_square_dimensions,height=plot_square_dimensions)
    y_title_text = r"$\text{Cross Section [cm}^{2}\text{] (normalized to nucleon)}$"

    y1 = go.layout.YAxis(title=y_title_text,
                        title_standoff = 0,
                        #range=[start_y_range,stop_y_range],
                        type="log",
                        titlefont=go.layout.yaxis.title.Font(color='SteelBlue'))

    x_title_text = r"$\text{WIMP Mass [GeV}/c^{2}]$"
    x1 = go.layout.XAxis(title=x_title_text,
                        title_standoff = 0,
                        type="log",
                        #type="linear",
                        #range=[start_x_range,stop_x_range],
                        titlefont=go.layout.xaxis.title.Font(color='SteelBlue'))


    ##title1=go.layout.Title(text="Dark Matter Detection Results")

    fig_chart_populated = go.Figure(
        data=[go.Scatter(x=[], y=[])],
        layout=go.Layout(
            margin=m1,
            yaxis= y1,
            xaxis= x1
        )
    )

    plot_title = plot_name_in + " - P: " + str(plot_id_in) + " - O: " + str(old_plot_id_in)

    fig_chart_populated.update_layout(
        title=dict(text=plot_title ,font=dict(size=16),automargin=True,yref='paper')
    )

    fig_chart_populated.update_layout(hw)

    for index, row in data_display_df_in.iterrows():
        #print(row['limit_id'])
        data_id_selected = row['data_id']
        #print('selected data_id >>', data_id_selected)
        #data_about_selected_df = self.data_about_df[self.data_about_df['data_id']==data_id_selected].copy()
        data_display_selected_df_in = data_display_df_in[data_display_df_in['data_id']==data_id_selected].copy()
        data_data_selected_df_in = data_data_df_in[data_data_df_in['data_id']==data_id_selected].copy()
        # df[(df.year == 2013) & (df.country.isin(['US', 'FR', 'ES']))]
        for index, row in data_display_selected_df_in.iterrows():

            #trace_features : name, spin_dependency, result_type, mode_value, line_color, line_width, line_format, fill_style, fill_color, opacity_style, marker_size, marker_color
            trace_data = data_data_selected_df_in[(data_data_selected_df_in['data_id']==row['data_id']) \
                                            & (data_data_selected_df_in['trace_id']==row['trace_id'])].copy()
            #print(trace_data)
            #print("trace_data >>>>" , trace_data)
            row_data = row.copy()
            trace_name = row_data['trace_name']
            #print(data_id_selected, trace_name )
            mode = row['mode_value']
            if row['line_format'] == 'none':
                line_style = 'solid'
            else:
                line_style = row['line_format']
            line_width = row['line_width']
            line_color = row['line_color']
            opacity = row['opacity_style']
            fill_color = row['fill_color']
            symbol = 'circle'
            if row['symbol'] == 'triangle':
                symbol = 'triangle-up'
                mode = 'lines+markers'
            elif row['symbol'] == 'pent':
                symbol = 'pentagon'
                mode = 'lines+markers'
            elif row['symbol'] == 'dot':
                symbol = 'circle'
                mode = 'lines+markers'
            elif row['symbol'] == 'none':
                mode = 'lines'
            else:
                symbol = row['symbol']
                mode = 'lines+markers'
            marker_size = row['marker_size']
            marker_color = row['marker_color']
            fill_style = row['fill_style']
            if fill_style != 'fill':
                fig_chart_populated.add_trace(go.Scatter(x=trace_data['masses'],
                                              y=trace_data['cross_sections'],
                                              name=trace_name,
                                              #line={'dash':line_style},
                                              mode=mode,
                                              #mode='lines', # 'lines' or 'markers'
                                              #mode=trace_mode_value,
                                              line=dict(width=line_width,dash=line_style,color=line_color),
                                              #line=dict(width=1,dash=None,color=row['line_color']),
                                              #fill='toself',
                                              #fill=None,
                                              #opacity=opacity,
                                              #fillcolor='blue',
                                              #fillcolor = fill_color,
                                              #marker_symbol=None,
                                              #marker = None,
                                              marker_symbol=symbol,
                                              #marker_symbol='square',
                                              marker=dict(size=marker_size,color=marker_color),
                                              #legendgroup=str(row['limit_id']),  # this can be any string, not just "group"
                                              #legendgrouptitle_text=str(row['limit_id']),
                                              #########
                                              hovertemplate =
                                                  '<i><b>Cross Section</b></i>: %{y:.4E}'+
                                                  '<br><b>Mass</b>: %{x}<br>',
                                              #########
                                              text=trace_name,
                                              showlegend=False,
                                                  ))
            else:
                ## filled trace

                ##
                '''
                mode=mode,
                                              #opacity=opacity,
                                              opacity=0.5,
                                              #fillcolor = f"'{fill_color}'",
                                              fillcolor = fill_color,
                                              fill='toself',
                                              #text=row[current_column],
                                              line=dict(width=4,dash=line_format,color=row['line_color']),
                '''
                ##
                opacity_in = 0.01
                #fill_string = f"\'rgba(%u,%u,%u,{opacity_in})\'" % (100, 200, 255)
                fill_string = f"rgba(%u,%u,%u,{opacity_in})" % (255, 0, 0)
                print('fill_string>>>>>>>>>>>>>', fill_string)

                fig_chart_populated.add_trace(go.Scatter(x=trace_data['masses'],
                                              y=trace_data['cross_sections'],
                                              name=trace_name,
                                              mode='lines', # 'lines' or 'markers'
                                              line=dict(width=line_width,color=fill_color),
                                              fill='toself',
                                              #opacity=opacity,
                                              #opacity=0.5,
                                              #fillcolor = "'" & fill_color & "'",
                                              #fillcolor='rgba(255, 0, 0, .1)',
                                              #fillcolor  = fill_string,
                                              fillcolor = fill_color,
                                              ######### '%{x},%{y}<br>' +
                                              hovertemplate =
                                                  '<i>Cross Section</i>: %{y:.4E}'+
                                                  '<br><b>Mass</b>: %{x}<br>',
                                              #########
                                              text=trace_name, ## this needs to be an array to appear in the hovertext
                                              showlegend=False,
                                                  ))
    return fig_chart_populated
fig_chart_1 = create_actual_plot(plot_name, plot_id, old_plot_id, data_display_df, data_data_df )
fig_chart_1.show()

# Create Legend

In [30]:
def create_populated_legend(data_display_df_in):
    rows_list = list(range(1,20))
    cols_list = list(range(1,4))

    table_rows=20
    table_cols=3
    #plot_square_dimensions = screen_height_in / 2
    legend_width = 800 ## this will be a maximum and will shrink if screen size < 800
    legend_height = 20 * 16
    fig_chart_legend = make_subplots(
                      #column_titles = ['data_id','data_label','line_format','fill'],
                      column_titles = ['data_id','data_label','format'],
                      rows=table_rows,
                      cols=table_cols,
                      horizontal_spacing = 0.00,
                      vertical_spacing = 0.00,
                      #subplot_titles=(titles)
                      column_widths=[0.1,0.8,0.1])

    fig_chart_legend.update_layout(
        #    autosize=False,
            width=legend_width,
            height=legend_height,
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0,
                pad=0
            ),
            paper_bgcolor="LightSteelBlue",
        )

    fig_chart_legend.update_xaxes(showgrid=False)
    fig_chart_legend.update_yaxes(showgrid=False)
    #legend
    fig_chart_legend.update_layout(showlegend=False)
    #x axis
    fig_chart_legend.update_xaxes(visible=False)
    #y axis
    fig_chart_legend.update_yaxes(visible=False)

    fig_chart_legend.data = []
    #fig_legend_out.show()

    # Any changes to the fig must be applied to the DataFrame as the dataframe
    # will be used when the plot is saved.
    # Saving zoom is still to be implemented

    #print("CD : data_display_df>>>>>>>>>", data_display_df_in)
    display_legend_df = data_display_df_in[['data_id','data_label',\
                                            'mode_value','opacity_style',\
                                            'line_format','line_width','line_color',\
                                            'fill_color','fill_style',\
                                            'symbol','symbol_color',\
                                            'marker_size','marker_color']].copy()

    display_legend_df.drop_duplicates(inplace=True)

    rowloop = 0
    for index, row in display_legend_df.iterrows():
        mode = row['mode_value']
        if row['line_format'] == 'none':
            line_style = 'solid'
        else:
            line_style = row['line_format']
        line_width = row['line_width']
        line_color = row['line_color']
        opacity = row['opacity_style']
        fill_color = row['fill_color']
        ## these may not be relevant and a Matlab translation remnant
        if row['symbol'] == 'triangle':
            symbol = 'triangle-up'
        elif row['symbol'] == 'pent':
            symbol = 'pentagon'
        elif row['symbol'] == 'dot':
            symbol = 'circle'
        elif row['symbol'] == 'none':
            symbol = 'circle'
        else:
            symbol = row['symbol']
        marker_size = row['marker_size']
        marker_color = row['marker_color']
        fill_style = row['fill_style']
        #print(row['c1'], row['c2'])
        rowloop +=1
        for c in cols_list: #enumerate here to get access to i
            # STEP 2, notice position of arguments!
            table_column_names = ['data_id','data_label','format']
            scatter_mode_list = ['text-number','text-text','format']
            #current_column = table_column_names[c-1]
            #current_mode = scatter_mode_list[c-1]
            current_column = table_column_names[c-1]
            current_mode = scatter_mode_list[c-1]
            line_format = 'solid'
            if current_mode =='format':
                if fill_style != 'none':
                    mode = 'lines'
                    fig_chart_legend.add_trace(go.Scatter(x=[0,1,1,0,0],
                                              y=[0,0,1,1,0],
                                              mode=mode,
                                              #opacity=opacity,
                                              opacity=0.5,
                                              #fillcolor = f"'{fill_color}'",
                                              fillcolor = fill_color,
                                              fill='toself',
                                              #text=row[current_column],
                                              line=dict(width=4,dash=line_format,color=row['line_color']),
                                            ),
                                  row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                                  col=c)
                else:
                    if row['line_format'] == 'none':
                        line_format = 'solid'
                    elif row['symbol'] != 'none':
                        mode = 'lines+markers'
                        line_format = 'solid'
                    else:
                        line_format = row['line_format']
                        mode = 'lines'
                    fig_chart_legend.add_trace(go.Scatter(x=[1,2,3],
                                              y=[1,1,1],
                                              mode=mode,
                                              marker_symbol=symbol,
                                              marker=dict(
                                                      size=10,
                                                      color=row['symbol_color'],#set color equal to a variable
                                                      #colorscale='Viridis', # one of plotly colorscales
                                                      showscale=False,
                                                      ),
                                              #text=row[current_column],
                                              line=dict(width=4,dash=line_format,color=row['line_color']),
                                            ),
                                  row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                                  col=c)
            if current_mode =='text-text':
                fig_chart_legend.add_trace(go.Scatter(x=[1,2],
                                          textposition='middle right',
                                          y=[1,1],
                                          mode='text',
                                          text=[row[current_column],'']
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)


            if current_mode =='text-number':
                fig_chart_legend.add_trace(go.Scatter(x=[1],
                                          textposition='middle right',
                                          y=[1],
                                          mode='text',
                                          text=row[current_column]
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)
            if current_mode =='markers':
                fig_chart_legend.add_trace(go.Scatter(x=[1],
                                        y=[1],
                                        mode=current_mode,
                                        #text=row[current_column],
                                        marker_symbol=symbol,
                                        marker=dict(
                                                  size=10,
                                                  color=row['symbol_color'],#set color equal to a variable
                                                  #colorscale='Viridis', # one of plotly colorscales
                                                  showscale=False,
                                                  )
                                        ),
                              row=rowloop, #index for the subplot, i+1 because plotly starts with 1
                              col=c)

            fig_chart_legend.update_xaxes(showgrid=False)
            fig_chart_legend.update_yaxes(showgrid=False)
            #legend
            fig_chart_legend.update_layout(showlegend=False)
            #x axis
            fig_chart_legend.update_xaxes(visible=False)
            #y axis
            fig_chart_legend.update_yaxes(visible=False)
    return fig_chart_legend

legend_fig = create_populated_legend(data_display_df)
legend_fig.show()